In [ ]:
!pip install "dask[complete]"
!pip install pyarrow pandas
!pip install mlxtend

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 20.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 206.6/206.6 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 42.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 56.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.3/43.3 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.9/83.9 kB 6.6 MB/s eta 0:00:00
  Attempting uninstall: importlib-metadata
    Found existing installation: importlib-metadata 4.6.4
    Uninstalling importlib-metadata-4.6.4:
      Successfully uninstalled importlib-metadata-4.6.4
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 5.8 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
from pathlib import Path
import os
import datetime
from datetime import timedelta
from mlxtend.frequent_patterns import fpgrowth
from mlxtend.frequent_patterns import association_rules
from mlxtend.frequent_patterns import apriori
from mlxtend.preprocessing import TransactionEncoder
import dask.dataframe as dd
import warnings
from sklearn.preprocessing import LabelEncoder
warnings.filterwarnings('ignore')

In [ ]:
data_dir = Path('/content/drive/MyDrive/Data_hasking')
ddf = dd.read_parquet(str(data_dir / '*.parquet'))
df = ddf.compute()

In [ ]:
pd.set_option('display.max_colwidth', None)

In [ ]:
df.head()

,CUST_CUSTNO,ACC_BUSINESSTYPE,ACC_ACCNO,ACC_BUSINESSNO,ACC_CURRENCYISO,BUSINESSNO_TRANS,FK_CURRENCY,AMOUNT,AMOUNTORIG,BRANCH_OFFICE,...,REASON4,TR_FLAG_01,ANALYTICAL_TRANS_CODE,TR_SP_01,TR_SP_02,TR_SP_03,TR_SP_04,TR_SP_05,TR_SP_10,VALUEDATE
0,a168d638771f6240e978956399d9f16621b02655b20ae81efcbeaf18af4cdc73,CARD,432a9ae3afb5774dbb60c0f091dbd08193eaa4161d08ef4fbcd6698608be9ff7,0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6,VND,202308011836871,VND,-9600.0,-9600.0,110000,...,<NA>,Y,TP137,3006ab7cb6dc7542968e1e22dc0167a1d5b40a5463091ab12252b579578f62b0,POS,HOI SO SHB,CM,<NA>,5590.91,20230801
1,4fd201b58b6b81ad3366160ff82f3d53f622e0290a57dbe0b94e10c5274c88df,CARD,3d1eaec79cbbaad7f4b8019b874ab4fe327f490e32ab4cd0ac1deab84f26df36,0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6,VND,202308011836872,VND,-100.0,-100.0,110000,...,<NA>,Y,TP900,afcd5e115702ec6f7e4ea75699f88062b34506ff0c9ce66df0dd52943494e89f,ATM,HOI SO SHB,CM,<NA>,6065.63,20230801
2,52dcdc6af49899ae1dfae36d4eee1625fe717ac6897c2b411a30512c3ffdd834,CARD,8c3b276a636351fae92fe15ec81a2c1835f54f394fea69352f98d033b9e480a1,0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6,VND,202308011836873,VND,-99.2,-99.2,110000,...,<NA>,Y,TP501,53f761a09578c10e92ec4378506860638bb0fd5b45e2c86c182fcbdbd30c0e0e,EPOS,HOI SO SHB,CM,<NA>,5425.42,20230801
3,db34f14574441e3f42d7ea1d5299204ea481f9ac8f5ff5fb8b90e981ba14fb4a,CARD,a7c0264cb2746c27921013f13b7326c2a1d4e2a7ec41ad1ed58ec3e303612fa9,0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6,VND,202308011836874,VND,-120.0,-120.0,110000,...,<NA>,Y,TP900,e5524afd37bd3778471ce72c4b1fc851928895d9cc15350e7d34a388bbd02084,ATM,HOI SO SHB,CM,<NA>,6061.19,20230801
4,49454b2d0968a2714066486a540ac8f0a57ee432cc3ca429fb5afc086bfeacd0,CARD,41f062e47151420fa96091222e6be31458d670f623edaef9d7a19a166d2de281,0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6,VND,202308011836875,VND,-120.0,-120.0,110000,...,<NA>,Y,TP900,3e3888614d3fce7b4899b7d28c8fed122efc90fcbd74c2108462bf92379915d6,ATM,HOI SO SHB,CM,<NA>,6013.04,20230801


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 139389125 entries, 0 to 1031063
Data columns (total 26 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   CUST_CUSTNO            string 
 1   ACC_BUSINESSTYPE       string 
 2   ACC_ACCNO              string 
 3   ACC_BUSINESSNO         string 
 4   ACC_CURRENCYISO        string 
 5   BUSINESSNO_TRANS       string 
 6   FK_CURRENCY            string 
 7   AMOUNT                 float64
 8   AMOUNTORIG             float64
 9   BRANCH_OFFICE          string 
 10  CONTRA_ACCNO           string 
 11  CONTRA_ZIP             string 
 12  CONTRA_NAME            string 
 13  CSHYN                  string 
 14  REASON1                string 
 15  REASON2                string 
 16  REASON4                string 
 17  TR_FLAG_01             string 
 18  ANALYTICAL_TRANS_CODE  string 
 19  TR_SP_01               string 
 20  TR_SP_02               string 
 21  TR_SP_03               string 
 22  TR_SP_04             

In [ ]:
valeurs_manquantes = df.isnull().sum()
valeurs_manquantes

CUST_CUSTNO                      0
ACC_BUSINESSTYPE           1325467
ACC_ACCNO                        0
ACC_BUSINESSNO                   0
ACC_CURRENCYISO            1325467
BUSINESSNO_TRANS                 0
FK_CURRENCY                      0
AMOUNT                           0
AMOUNTORIG                       0
BRANCH_OFFICE                    0
CONTRA_ACCNO                     0
CONTRA_ZIP               128047268
CONTRA_NAME              128043726
CSHYN                            0
REASON1                     717101
REASON2                   37025377
REASON4                  115787808
TR_FLAG_01                       0
ANALYTICAL_TRANS_CODE            0
TR_SP_01                         0
TR_SP_02                         0
TR_SP_03                         0
TR_SP_04                         0
TR_SP_05                 128043726
TR_SP_10                 139257171
VALUEDATE                        0
dtype: int64

In [ ]:
for column in df.columns:
    if pd.api.types.is_string_dtype(df[column]):
        df[column] = df[column].fillna('')
    elif pd.api.types.is_numeric_dtype(df[column]):
        df[column] = df[column].fillna(0)

In [ ]:
valeurs_manquantes = df.isnull().sum()
valeurs_manquantes

CUST_CUSTNO              0
ACC_BUSINESSTYPE         0
ACC_ACCNO                0
ACC_BUSINESSNO           0
ACC_CURRENCYISO          0
BUSINESSNO_TRANS         0
FK_CURRENCY              0
AMOUNT                   0
AMOUNTORIG               0
BRANCH_OFFICE            0
CONTRA_ACCNO             0
CONTRA_ZIP               0
CONTRA_NAME              0
CSHYN                    0
REASON1                  0
REASON2                  0
REASON4                  0
TR_FLAG_01               0
ANALYTICAL_TRANS_CODE    0
TR_SP_01                 0
TR_SP_02                 0
TR_SP_03                 0
TR_SP_04                 0
TR_SP_05                 0
TR_SP_10                 0
VALUEDATE                0
dtype: int64

In [ ]:

# Convert VALUEDATE column to datetime (if not already converted)
ddf['VALUEDATE'] = dd.to_datetime(ddf['VALUEDATE'], errors='coerce')

# Filter using Dask operations
date_filter = (ddf['VALUEDATE'] >= '2023-12-03') & (ddf['VALUEDATE'] <= '2023-12-07')
ddf_filtered = ddf[date_filter]

# Compute the filtered DataFrame to get a Pandas DataFrame
df_filtered = ddf_filtered.compute()

# Display filtered data
df_filtered

,CUST_CUSTNO,ACC_BUSINESSTYPE,ACC_ACCNO,ACC_BUSINESSNO,ACC_CURRENCYISO,BUSINESSNO_TRANS,FK_CURRENCY,AMOUNT,AMOUNTORIG,BRANCH_OFFICE,...,REASON4,TR_FLAG_01,ANALYTICAL_TRANS_CODE,TR_SP_01,TR_SP_02,TR_SP_03,TR_SP_04,TR_SP_05,TR_SP_10,VALUEDATE
0,1655bf1b772d90df34c51ba9240f683d3847700b330295a077d17e40491d1c28,CASA,0c643de92622a5f3bdb856d4471510b234be53fb0ab018c4ee971bf185b4b362,0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6,VND,2023120391980769,VND,-650.0,-650.0,110000,...,<NA>,N,1|2,e4180c1cc088ebea7f7a882144f55159a8dddad215334fec1f741f658bfcbc1a,MOB,HOI SO SHB,FP,<NA>,NaN,2023-12-03
1,1655bf1b772d90df34c51ba9240f683d3847700b330295a077d17e40491d1c28,CASA,0c643de92622a5f3bdb856d4471510b234be53fb0ab018c4ee971bf185b4b362,0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6,VND,2023120391467506,VND,-34.0,-34.0,110000,...,<NA>,N,1|2,b403758c4a7b9fbc934464efaaaab53c7e24f0b68891f78329e964152de23684,MOB,HOI SO SHB,FP,<NA>,NaN,2023-12-03
2,1b781f92156f274627a131e707e7a890548aea921b84f5ddec59322ebc61e86d,CASA,df774ebe64129d3a6cf3af75e7de8d01ca85e92b2f2c1c44216c782d07183742,0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6,VND,2023120391730083,VND,120000.0,120000.0,110000,...,<NA>,Y,1|1,22527ba60412b7170a3dfcd831805f966b244b354d1607a33d6b0efdeb592bac,NET,HOI SO SHB,FP,<NA>,NaN,2023-12-03
3,53048c1afff7f2cff6e1469fd1659f99ec93696df7a3ce309aef278bfed89856,CASA,2715ec2c8fc66cc32a6accc328d63843fb0dcd5565cd590da6421ddfd950b810,0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6,VND,2023120391677652,VND,-62.0,-62.0,110000,...,<NA>,N,1|2,fb05380821892df14964a30cbe3fd795e0cb225c8da13b3385ddb7b375241f15,MOB,HOI SO SHB,FP,<NA>,NaN,2023-12-03
4,5abad7746ef69e998247ba7848ca6224dd106e300182370090564f0cae691746,CASA,a5dc6ed17504c973d61d1ea548eb1945ae698847f3cb5c9fe44c5388cf45b807,0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6,VND,2023120391519660,VND,-1500.0,-1500.0,110000,...,<NA>,N,1|2,54ff7b7a4f04b0a6e7de5abd7a4473ec7831e5579431a35bd21fceddaad0a1fb,MOB,HOI SO SHB,FP,<NA>,NaN,2023-12-03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
802043,b4bb40c42e955a98f1641564f944c8b630b33307e680428e76fb37322d71caf1,<NA>,0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6,0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6,<NA>,2023120794685820,VND,4750.0,4750.0,110000,...,<NA>,N,1|2,98be8d28c01235ba77ff244ca6035dd86086b1ca28ca801a4fca515f57b3b9d6,MOB,HOI SO SHB,A2,VU THI YEN,NaN,2023-12-07
802044,b4bb40c42e955a98f1641564f944c8b630b33307e680428e76fb37322d71caf1,<NA>,0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6,0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6,<NA>,2023120794734517,VND,-3500.0,-3500.0,110000,...,<NA>,N,1|2,4dc1194ff047ca5307f6848f4b69aa562c00e6022e5413205404eb622c803a6a,MOB,HOI SO SHB,FP,<NA>,NaN,2023-12-07
802045,b4bb40c42e955a98f1641564f944c8b630b33307e680428e76fb37322d71caf1,<NA>,0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6,0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6,<NA>,2023120794734516,VND,900.0,900.0,110000,...,<NA>,Y,1|1,983fc2ba457d77883ea2a5160e77b81ffb98dc6e325abec2907a4ed8e43ec90c,NET,HOI SO SHB,FP,<NA>,NaN,2023-12-07
802046,b4bb40c42e955a98f1641564f944c8b630b33307e680428e76fb37322d71caf1,<NA>,0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6,0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6,<NA>,2023120794734518,VND,-850.0,-850.0,110000,...,<NA>,N,1|2,38213e45e4a821d58cc7f7ca3294ba14479168f67bf22ea53ca5a15e2eb8bc60,MOB,HOI SO SHB,FP,<NA>,NaN,2023-12-07


In [ ]:

# Convert VALUEDATE column to datetime (if not already converted)
ddf['VALUEDATE'] = dd.to_datetime(ddf['VALUEDATE'], errors='coerce')

# Filter using Dask operations
date_filter = (ddf['VALUEDATE'] >= '2023-10-04') & (ddf['VALUEDATE'] <= '2023-10-08')
ddf_filtered = ddf[date_filter]

# Compute the filtered DataFrame to get a Pandas DataFrame
df_filtered_rule2 = ddf_filtered.compute()

# Display filtered data
df_filtered_rule2

,CUST_CUSTNO,ACC_BUSINESSTYPE,ACC_ACCNO,ACC_BUSINESSNO,ACC_CURRENCYISO,BUSINESSNO_TRANS,FK_CURRENCY,AMOUNT,AMOUNTORIG,BRANCH_OFFICE,...,REASON4,TR_FLAG_01,ANALYTICAL_TRANS_CODE,TR_SP_01,TR_SP_02,TR_SP_03,TR_SP_04,TR_SP_05,TR_SP_10,VALUEDATE
0,311c0c8d1db740e6963d4f0563b10e917dd6055c858aa6d9b474763967231c62,CARD,13525def187ae4928d50d895f0d72f02fda1d8151e65ddafa3b7154d197918c5,0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6,VND,2023100445026483,VND,-9.90,-9.90,110000,...,<NA>,Y,TP702,6bdf5b10622aad54c4cdad0e864eb4ff8b66a4657801057ced52ea121ca0844e,BRC,HOI SO SHB,CM,<NA>,NaN,2023-10-04
1,3a55c0eca9da66d139ba5cb50c8dac76f9c7409058062e67379d6451552cb749,CARD,078bc5215458eea1f474eba8781bce260d336ceee58343a98af89397e85b1bb4,0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6,VND,2023100445026484,VND,-9.90,-9.90,110000,...,<NA>,Y,TP702,082aee06256a9a429edab523cfa53969d0f1838d43bdd6ed1bde008f8d48ee4b,BRC,HOI SO SHB,CM,<NA>,NaN,2023-10-04
2,16c32a971c270244dde7595c8e09cd73e492f903fffa2db5ecf214f566971195,CARD,13f85ff4083534947e4a29d4a1949b8ebc3d14b0a1ed08414998ee41c886ea71,0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6,VND,2023100445026485,VND,-9.90,-9.90,110000,...,<NA>,Y,TP702,2ba2abf7267cdb849f115486157736c1e99eddcbb7109a22138e124f79129794,BRC,HOI SO SHB,CM,<NA>,NaN,2023-10-04
3,54c8b93404ae63d7a95118bd848744225bbee6a6458c0a61e74734eb303611b7,CARD,ec003fbafa4d738531e60f73bbb1c8b8909823914579a4a89dc3e025909dd786,0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6,VND,2023100445026486,VND,-9.90,-9.90,110000,...,<NA>,Y,TP702,7b2846a6ae6aecad77586ab47e78e1a20288a447266ef33d9e29158c13ee2f21,BRC,HOI SO SHB,CM,<NA>,NaN,2023-10-04
4,f61b11e7cb85da92f6df79ed4d52b31e7fad70062f5a7c7accd507ce40873d56,CARD,16aaa93dcaf0539c8709ffd6afa72f87bc44ce8ffb7281c8c61f79fda0903880,0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6,VND,2023100445026487,VND,-9.90,-9.90,110000,...,<NA>,Y,TP702,0a46ef6aa779d0df0f47a7dac60ceaff1ef0052a5aae1f1f07a85f6732ce39d8,BRC,HOI SO SHB,CM,<NA>,NaN,2023-10-04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
610926,cdc586b6d0a900e1b2ed82fab621f1a8a7cff161ad040d853007aa269dd79988,<NA>,0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6,0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6,<NA>,2023100847552882,VND,5000.00,5000.00,110000,...,<NA>,N,1|1,cdea7a80a12fd888b9b435fad6537f153ecc373f136d03029e4000449d28e97a,MOB,HOI SO SHB,FP,<NA>,NaN,2023-10-08
610927,4f2ef31d6ea9f5ada17fde3e1a71942805cef02868a5c951530ff01f9a3bad42,<NA>,0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6,0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6,<NA>,2023100847557881,VND,202.26,202.26,110000,...,<NA>,Y,1|1,07d05cf8d73d98cd60594896d3c6c9e02f233a70b158f385d1d9d3d43a454112,NET,HOI SO SHB,FP,<NA>,NaN,2023-10-08
610928,5e0b1cc49f2b8e63105596aab97a4ed2d2f6706e7196a5ee76a16a22d73003be,<NA>,0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6,0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6,<NA>,2023100847617237,VND,-1080.00,-1080.00,110000,...,<NA>,N,1|2,730353e8e24132d45fd1045a855e3737dc4d16f3c0779372dce7ea3da162391e,MOB,HOI SO SHB,FP,<NA>,NaN,2023-10-08
610929,5e0b1cc49f2b8e63105596aab97a4ed2d2f6706e7196a5ee76a16a22d73003be,<NA>,0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6,0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6,<NA>,2023100847617238,VND,444.00,444.00,110000,...,<NA>,Y,1|1,afcd089c85e1e56701f8f1c8a90c9f497e26ebad4929ba139775f6a2af3251ec,NET,HOI SO SHB,FP,<NA>,NaN,2023-10-08


In [ ]:

# Convert VALUEDATE column to datetime (if not already converted)
ddf['VALUEDATE'] = dd.to_datetime(ddf['VALUEDATE'], errors='coerce')

# Filter using Dask operations
date_filter = (ddf['VALUEDATE'] >= '2024-01-14') & (ddf['VALUEDATE'] <= '2024-01-18')
ddf_filtered = ddf[date_filter]

# Compute the filtered DataFrame to get a Pandas DataFrame
df_filtered_rule3 = ddf_filtered.compute()

# Display filtered data
df_filtered_rule3

,CUST_CUSTNO,ACC_BUSINESSTYPE,ACC_ACCNO,ACC_BUSINESSNO,ACC_CURRENCYISO,BUSINESSNO_TRANS,FK_CURRENCY,AMOUNT,AMOUNTORIG,BRANCH_OFFICE,...,REASON4,TR_FLAG_01,ANALYTICAL_TRANS_CODE,TR_SP_01,TR_SP_02,TR_SP_03,TR_SP_04,TR_SP_05,TR_SP_10,VALUEDATE
0,4ea51a8df990ffb5b13a4d69aa832a48e2172bbe97465723c1f1fe6a4a0e5dcc,CARD,b04b0fd1a4d544cc5071484e1e134662d80e72658a578eed08b251f7feb33b92,0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6,VND,2024011427275035,VND,262.47,262.47,110000,...,<NA>,Y,TP705,12e2a1f3aa28eb6af324a40d86d3fffcd9addace59a9b7388a4b917b7b9cc277,VOICE,HOI SO SHB,TP,<NA>,6013.45,2024-01-14
1,4ea51a8df990ffb5b13a4d69aa832a48e2172bbe97465723c1f1fe6a4a0e5dcc,CARD,b04b0fd1a4d544cc5071484e1e134662d80e72658a578eed08b251f7feb33b92,0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6,VND,2024011427275036,VND,262.47,262.47,110000,...,<NA>,Y,TP705,86093a45d0899c37b0542af660e6b8ad3164d9a8dd6050d9deff4f00ab7b0c7e,VOICE,HOI SO SHB,TP,<NA>,6022.11,2024-01-14
2,54f6cacfc0eaf9af95158c6045e33644cdc3452fda540dc6de70b93a6ea5fd8e,CARD,8bb025cb698b6852e84003adc7a2e1f6863e3eeff08f509964a5884286edcfce,0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6,VND,2024011427275037,VND,-75000.00,-75000.00,110000,...,<NA>,Y,TP137,93e1b5fe42bf194f4d9593aab22bf5ba50cf0c2664d16369bf8ca5ac72615116,POS,HOI SO SHB,CM,<NA>,6577.18,2024-01-14
3,eaf72624d3aaaae6e216516b3085e9530710419da614da83015204de7216a9eb,CARD,edc9c2c7bee1704c8e65c3d6d53eebd15f2b5939fddde7bad04950eafa2a9815,0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6,VND,2024011427275038,VND,-20690.00,-20690.00,110000,...,<NA>,Y,TP137,2db475d15d973ae0cfba41c5cac875eb48340db52d1ba734459ca5e402b12b0c,POS,HOI SO SHB,CM,<NA>,5749.68,2024-01-14
4,112677a05c22e0272ef07ba0dce4236e207642b2f31c213ca7b3b0f8beb94a09,CARD,861ddd092f5c3291f97fe46481da67ad6472ad4f9129daddb133b7603584a75e,0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6,VND,2024011427275039,VND,2527.47,2527.47,110000,...,<NA>,Y,TP705,fdf4986be74ba753a757434c0e05410474a253600e13a75555d936b969e0335d,VOICE,HOI SO SHB,TP,<NA>,6023.23,2024-01-14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
963128,07063e278b269237a9516eb99830764c5c34f9c377fbaa29f68db5f1dd0e46a9,<NA>,0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6,0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6,<NA>,2024011830932333,VND,100.00,100.00,110000,...,T tu XXXXXXXXXXXXXXXX NGUYE,Y,1|1,dbd0f0999885aba9c8202a16773ca20203c9c3525408bb25607ff0b1d4c766bb,NET,HOI SO SHB,FP,<NA>,NaN,2024-01-18
963129,07063e278b269237a9516eb99830764c5c34f9c377fbaa29f68db5f1dd0e46a9,<NA>,0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6,0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6,<NA>,2024011830932334,VND,545.00,545.00,110000,...,<NA>,Y,1|1,d914722725f4537d65830411c5b538307c1ab8717f63314326642f6822577061,NET,HOI SO SHB,FP,<NA>,NaN,2024-01-18
963130,07063e278b269237a9516eb99830764c5c34f9c377fbaa29f68db5f1dd0e46a9,<NA>,0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6,0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6,<NA>,2024011830932335,VND,330.00,330.00,110000,...,),Y,1|1,443f7870719ced1e7263cb5a8ff8ee5a4a4315cc1f3eed1c675310a7d7a102fe,NET,HOI SO SHB,FP,<NA>,NaN,2024-01-18
963131,07063e278b269237a9516eb99830764c5c34f9c377fbaa29f68db5f1dd0e46a9,<NA>,0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6,0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6,<NA>,2024011830932336,VND,290.00,290.00,110000,...,<NA>,Y,1|1,0f3082536836099b0668de9f1877a3a89f00f853f11060286bb331a13769cb09,NET,HOI SO SHB,FP,<NA>,NaN,2024-01-18


In [ ]:
# prompt: merge the 3 datasets filtered

import pandas as pd
merged_data = pd.concat([df_filtered, df_filtered_rule2, df_filtered_rule3], ignore_index=True)
merged_data


,CUST_CUSTNO,ACC_BUSINESSTYPE,ACC_ACCNO,ACC_BUSINESSNO,ACC_CURRENCYISO,BUSINESSNO_TRANS,FK_CURRENCY,AMOUNT,AMOUNTORIG,BRANCH_OFFICE,...,REASON4,TR_FLAG_01,ANALYTICAL_TRANS_CODE,TR_SP_01,TR_SP_02,TR_SP_03,TR_SP_04,TR_SP_05,TR_SP_10,VALUEDATE
0,1655bf1b772d90df34c51ba9240f683d3847700b330295a077d17e40491d1c28,CASA,0c643de92622a5f3bdb856d4471510b234be53fb0ab018c4ee971bf185b4b362,0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6,VND,2023120391980769,VND,-650.0,-650.0,110000,...,<NA>,N,1|2,e4180c1cc088ebea7f7a882144f55159a8dddad215334fec1f741f658bfcbc1a,MOB,HOI SO SHB,FP,<NA>,NaN,2023-12-03
1,1655bf1b772d90df34c51ba9240f683d3847700b330295a077d17e40491d1c28,CASA,0c643de92622a5f3bdb856d4471510b234be53fb0ab018c4ee971bf185b4b362,0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6,VND,2023120391467506,VND,-34.0,-34.0,110000,...,<NA>,N,1|2,b403758c4a7b9fbc934464efaaaab53c7e24f0b68891f78329e964152de23684,MOB,HOI SO SHB,FP,<NA>,NaN,2023-12-03
2,1b781f92156f274627a131e707e7a890548aea921b84f5ddec59322ebc61e86d,CASA,df774ebe64129d3a6cf3af75e7de8d01ca85e92b2f2c1c44216c782d07183742,0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6,VND,2023120391730083,VND,120000.0,120000.0,110000,...,<NA>,Y,1|1,22527ba60412b7170a3dfcd831805f966b244b354d1607a33d6b0efdeb592bac,NET,HOI SO SHB,FP,<NA>,NaN,2023-12-03
3,53048c1afff7f2cff6e1469fd1659f99ec93696df7a3ce309aef278bfed89856,CASA,2715ec2c8fc66cc32a6accc328d63843fb0dcd5565cd590da6421ddfd950b810,0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6,VND,2023120391677652,VND,-62.0,-62.0,110000,...,<NA>,N,1|2,fb05380821892df14964a30cbe3fd795e0cb225c8da13b3385ddb7b375241f15,MOB,HOI SO SHB,FP,<NA>,NaN,2023-12-03
4,5abad7746ef69e998247ba7848ca6224dd106e300182370090564f0cae691746,CASA,a5dc6ed17504c973d61d1ea548eb1945ae698847f3cb5c9fe44c5388cf45b807,0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6,VND,2023120391519660,VND,-1500.0,-1500.0,110000,...,<NA>,N,1|2,54ff7b7a4f04b0a6e7de5abd7a4473ec7831e5579431a35bd21fceddaad0a1fb,MOB,HOI SO SHB,FP,<NA>,NaN,2023-12-03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12332483,07063e278b269237a9516eb99830764c5c34f9c377fbaa29f68db5f1dd0e46a9,<NA>,0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6,0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6,<NA>,2024011830932333,VND,100.0,100.0,110000,...,T tu XXXXXXXXXXXXXXXX NGUYE,Y,1|1,dbd0f0999885aba9c8202a16773ca20203c9c3525408bb25607ff0b1d4c766bb,NET,HOI SO SHB,FP,<NA>,NaN,2024-01-18
12332484,07063e278b269237a9516eb99830764c5c34f9c377fbaa29f68db5f1dd0e46a9,<NA>,0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6,0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6,<NA>,2024011830932334,VND,545.0,545.0,110000,...,<NA>,Y,1|1,d914722725f4537d65830411c5b538307c1ab8717f63314326642f6822577061,NET,HOI SO SHB,FP,<NA>,NaN,2024-01-18
12332485,07063e278b269237a9516eb99830764c5c34f9c377fbaa29f68db5f1dd0e46a9,<NA>,0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6,0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6,<NA>,2024011830932335,VND,330.0,330.0,110000,...,),Y,1|1,443f7870719ced1e7263cb5a8ff8ee5a4a4315cc1f3eed1c675310a7d7a102fe,NET,HOI SO SHB,FP,<NA>,NaN,2024-01-18
12332486,07063e278b269237a9516eb99830764c5c34f9c377fbaa29f68db5f1dd0e46a9,<NA>,0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6,0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6,<NA>,2024011830932336,VND,290.0,290.0,110000,...,<NA>,Y,1|1,0f3082536836099b0668de9f1877a3a89f00f853f11060286bb331a13769cb09,NET,HOI SO SHB,FP,<NA>,NaN,2024-01-18


In [ ]:
#save the merged data

merged_data.to_parquet('/content/drive/MyDrive/merged_data.parquet')


In [ ]:

merged_data = pd.read_parquet('/content/drive/MyDrive/merged_data.parquet')
merged_data.head()


,CUST_CUSTNO,ACC_BUSINESSTYPE,ACC_ACCNO,ACC_BUSINESSNO,ACC_CURRENCYISO,BUSINESSNO_TRANS,FK_CURRENCY,AMOUNT,AMOUNTORIG,BRANCH_OFFICE,...,REASON4,TR_FLAG_01,ANALYTICAL_TRANS_CODE,TR_SP_01,TR_SP_02,TR_SP_03,TR_SP_04,TR_SP_05,TR_SP_10,VALUEDATE
0,1655bf1b772d90df34c51ba9240f683d3847700b330295a077d17e40491d1c28,CASA,0c643de92622a5f3bdb856d4471510b234be53fb0ab018c4ee971bf185b4b362,0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6,VND,2023120391980769,VND,-650.0,-650.0,110000,...,<NA>,N,1|2,e4180c1cc088ebea7f7a882144f55159a8dddad215334fec1f741f658bfcbc1a,MOB,HOI SO SHB,FP,<NA>,NaN,2023-12-03
1,1655bf1b772d90df34c51ba9240f683d3847700b330295a077d17e40491d1c28,CASA,0c643de92622a5f3bdb856d4471510b234be53fb0ab018c4ee971bf185b4b362,0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6,VND,2023120391467506,VND,-34.0,-34.0,110000,...,<NA>,N,1|2,b403758c4a7b9fbc934464efaaaab53c7e24f0b68891f78329e964152de23684,MOB,HOI SO SHB,FP,<NA>,NaN,2023-12-03
2,1b781f92156f274627a131e707e7a890548aea921b84f5ddec59322ebc61e86d,CASA,df774ebe64129d3a6cf3af75e7de8d01ca85e92b2f2c1c44216c782d07183742,0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6,VND,2023120391730083,VND,120000.0,120000.0,110000,...,<NA>,Y,1|1,22527ba60412b7170a3dfcd831805f966b244b354d1607a33d6b0efdeb592bac,NET,HOI SO SHB,FP,<NA>,NaN,2023-12-03
3,53048c1afff7f2cff6e1469fd1659f99ec93696df7a3ce309aef278bfed89856,CASA,2715ec2c8fc66cc32a6accc328d63843fb0dcd5565cd590da6421ddfd950b810,0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6,VND,2023120391677652,VND,-62.0,-62.0,110000,...,<NA>,N,1|2,fb05380821892df14964a30cbe3fd795e0cb225c8da13b3385ddb7b375241f15,MOB,HOI SO SHB,FP,<NA>,NaN,2023-12-03
4,5abad7746ef69e998247ba7848ca6224dd106e300182370090564f0cae691746,CASA,a5dc6ed17504c973d61d1ea548eb1945ae698847f3cb5c9fe44c5388cf45b807,0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6,VND,2023120391519660,VND,-1500.0,-1500.0,110000,...,<NA>,N,1|2,54ff7b7a4f04b0a6e7de5abd7a4473ec7831e5579431a35bd21fceddaad0a1fb,MOB,HOI SO SHB,FP,<NA>,NaN,2023-12-03


In [ ]:
# prompt: read it with dask library

import dask.dataframe as dd

# Read the parquet file using Dask
ddf = dd.read_parquet('/content/drive/MyDrive/merged_data.parquet')
df = ddf.compute()
df

,CUST_CUSTNO,ACC_BUSINESSTYPE,ACC_ACCNO,ACC_BUSINESSNO,ACC_CURRENCYISO,BUSINESSNO_TRANS,FK_CURRENCY,AMOUNT,AMOUNTORIG,BRANCH_OFFICE,...,REASON4,TR_FLAG_01,ANALYTICAL_TRANS_CODE,TR_SP_01,TR_SP_02,TR_SP_03,TR_SP_04,TR_SP_05,TR_SP_10,VALUEDATE
0,1655bf1b772d90df34c51ba9240f683d3847700b330295a077d17e40491d1c28,CASA,0c643de92622a5f3bdb856d4471510b234be53fb0ab018c4ee971bf185b4b362,0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6,VND,2023120391980769,VND,-650.0,-650.0,110000,...,<NA>,N,1|2,e4180c1cc088ebea7f7a882144f55159a8dddad215334fec1f741f658bfcbc1a,MOB,HOI SO SHB,FP,<NA>,NaN,2023-12-03
1,1655bf1b772d90df34c51ba9240f683d3847700b330295a077d17e40491d1c28,CASA,0c643de92622a5f3bdb856d4471510b234be53fb0ab018c4ee971bf185b4b362,0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6,VND,2023120391467506,VND,-34.0,-34.0,110000,...,<NA>,N,1|2,b403758c4a7b9fbc934464efaaaab53c7e24f0b68891f78329e964152de23684,MOB,HOI SO SHB,FP,<NA>,NaN,2023-12-03
2,1b781f92156f274627a131e707e7a890548aea921b84f5ddec59322ebc61e86d,CASA,df774ebe64129d3a6cf3af75e7de8d01ca85e92b2f2c1c44216c782d07183742,0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6,VND,2023120391730083,VND,120000.0,120000.0,110000,...,<NA>,Y,1|1,22527ba60412b7170a3dfcd831805f966b244b354d1607a33d6b0efdeb592bac,NET,HOI SO SHB,FP,<NA>,NaN,2023-12-03
3,53048c1afff7f2cff6e1469fd1659f99ec93696df7a3ce309aef278bfed89856,CASA,2715ec2c8fc66cc32a6accc328d63843fb0dcd5565cd590da6421ddfd950b810,0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6,VND,2023120391677652,VND,-62.0,-62.0,110000,...,<NA>,N,1|2,fb05380821892df14964a30cbe3fd795e0cb225c8da13b3385ddb7b375241f15,MOB,HOI SO SHB,FP,<NA>,NaN,2023-12-03
4,5abad7746ef69e998247ba7848ca6224dd106e300182370090564f0cae691746,CASA,a5dc6ed17504c973d61d1ea548eb1945ae698847f3cb5c9fe44c5388cf45b807,0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6,VND,2023120391519660,VND,-1500.0,-1500.0,110000,...,<NA>,N,1|2,54ff7b7a4f04b0a6e7de5abd7a4473ec7831e5579431a35bd21fceddaad0a1fb,MOB,HOI SO SHB,FP,<NA>,NaN,2023-12-03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
798147,07063e278b269237a9516eb99830764c5c34f9c377fbaa29f68db5f1dd0e46a9,<NA>,0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6,0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6,<NA>,2024011830932333,VND,100.0,100.0,110000,...,T tu XXXXXXXXXXXXXXXX NGUYE,Y,1|1,dbd0f0999885aba9c8202a16773ca20203c9c3525408bb25607ff0b1d4c766bb,NET,HOI SO SHB,FP,<NA>,NaN,2024-01-18
798148,07063e278b269237a9516eb99830764c5c34f9c377fbaa29f68db5f1dd0e46a9,<NA>,0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6,0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6,<NA>,2024011830932334,VND,545.0,545.0,110000,...,<NA>,Y,1|1,d914722725f4537d65830411c5b538307c1ab8717f63314326642f6822577061,NET,HOI SO SHB,FP,<NA>,NaN,2024-01-18
798149,07063e278b269237a9516eb99830764c5c34f9c377fbaa29f68db5f1dd0e46a9,<NA>,0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6,0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6,<NA>,2024011830932335,VND,330.0,330.0,110000,...,),Y,1|1,443f7870719ced1e7263cb5a8ff8ee5a4a4315cc1f3eed1c675310a7d7a102fe,NET,HOI SO SHB,FP,<NA>,NaN,2024-01-18
798150,07063e278b269237a9516eb99830764c5c34f9c377fbaa29f68db5f1dd0e46a9,<NA>,0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6,0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6,<NA>,2024011830932336,VND,290.0,290.0,110000,...,<NA>,Y,1|1,0f3082536836099b0668de9f1877a3a89f00f853f11060286bb331a13769cb09,NET,HOI SO SHB,FP,<NA>,NaN,2024-01-18


In [ ]:
label_encoders = {}
list_columns = ['ACC_BUSINESSTYPE', 'ACC_CURRENCYISO', 'TR_SP_01', 'TR_SP_02', 'TR_SP_03', 'TR_SP_04', 'TR_SP_05', 'TR_SP_10']

In [ ]:
for column in list_columns:
    le = LabelEncoder()
    df[column] = le.fit_transform(df[column])
    label_encoders[column] = le

In [ ]:

string_columns = [col for col in df.columns if pd.api.types.is_string_dtype(df[col])]
string_columns_without_list = [col for col in string_columns if col not in list_columns]
string_columns_without_list


In [ ]:
label_encoders = {}

for column in string_columns_without_list:
    le = LabelEncoder()
    df[column] = le.fit_transform(df[column])
    label_encoders[column] = le

In [ ]:
df.head()

In [ ]:
# Apply FP-Growth algorithm
frequent_itemsets= fpgrowth(df.applymap(lambda x: 1 if x else 0), min_support=0.9, use_colnames=True)

In [ ]:
# lower the value of confidence

rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.8)
rules


In [ ]:
# reverse encoding label

for column, le in label_encoders.items():
    df[column] = le.inverse_transform(df[column])
df.head()

In [ ]:
# prompt: valide some rules with examples from the data

# Select a rule from the filtered rules DataFrame
selected_rule = filtered_rules.iloc[0]

# Extract the antecedents and consequents of the selected rule
antecedents = selected_rule["antecedents"]
consequents = selected_rule["consequents"]

# Find examples from the data that satisfy the antecedents
antecedent_examples = df[df[list(antecedents)].all(axis=1)]

# Check if the consequents are also present in the examples
consequent_examples = antecedent_examples[antecedent_examples[list(consequents)].all(axis=1)]

# Print the examples
print("Examples where the antecedents and consequents are both present:")
consequent_examples


In [ ]:
# prompt: # Create transactions from the dataframe

import pandas as pd
# Create a transaction encoder object
te = TransactionEncoder()

# Convert the DataFrame to a transaction array
transactions = te.fit_transform(df)

# Convert the transaction array to a DataFrame
transactions_df = pd.DataFrame(transactions, columns=te.columns_)

# Print the transaction DataFrame
print(transactions_df)


In [ ]:
# prompt: # Example: using selected columns for transactions

import pandas as pd
# Create a list of selected columns for transactions
selected_columns = ['ACC_BUSINESSTYPE', 'ACC_CURRENCYISO', 'TR_SP_01', 'TR_SP_02', 'TR_SP_03', 'TR_SP_04', 'TR_SP_05', 'TR_SP_10']

# Create a transaction encoder object
te = TransactionEncoder()

# Convert the selected columns to a transaction array
transactions = te.fit_transform(df[selected_columns])

# Convert the transaction array to a DataFrame
transactions_df = pd.DataFrame(transactions, columns=te.columns_)

# Print the transaction DataFrame
print(transactions_df)


In [ ]:
# prompt: Apply FP-Growth Algorithm

frequent_itemsets = fpgrowth(transactions_df, min_support=0.1, use_colnames=True)
print(frequent_itemsets)
